In [113]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd

In [4]:
data = load_digits()

In [67]:
images_train, images_val, labels_train, labels_val = train_test_split(data.get('images'), data.get('target'))

In [72]:
images = images_train
img_shape = images[0].shape
images = images / tf.reduce_max(images)
images = tf.data.Dataset.from_tensor_slices(images)

labels = labels_train
n_labels = len(set(labels))
labels = tf.keras.utils.to_categorical(labels, num_classes=n_labels)
labels = tf.data.Dataset.from_tensor_slices(labels)

dataset = tf.data.Dataset.zip((images, labels)).batch(4)

In [71]:
images_val = images_val / tf.reduce_max(images_val)
images_val = tf.data.Dataset.from_tensor_slices(images_val)

labels_val = tf.keras.utils.to_categorical(labels_val, num_classes=n_labels)
labels_val = tf.data.Dataset.from_tensor_slices(labels_val)

dataset_val = tf.data.Dataset.zip((images_val, labels_val)).batch(4)

In [61]:
it = iter(dataset)
for i in range(3):
    print(next(it))

(<tf.Tensor: shape=(4, 8, 8), dtype=float64, numpy=
array([[[0.    , 0.    , 0.3125, 0.8125, 0.5625, 0.0625, 0.    , 0.    ],
        [0.    , 0.    , 0.8125, 0.9375, 0.625 , 0.9375, 0.3125, 0.    ],
        [0.    , 0.1875, 0.9375, 0.125 , 0.    , 0.6875, 0.5   , 0.    ],
        [0.    , 0.25  , 0.75  , 0.    , 0.    , 0.5   , 0.5   , 0.    ],
        [0.    , 0.3125, 0.5   , 0.    , 0.    , 0.5625, 0.5   , 0.    ],
        [0.    , 0.25  , 0.6875, 0.    , 0.0625, 0.75  , 0.4375, 0.    ],
        [0.    , 0.125 , 0.875 , 0.3125, 0.625 , 0.75  , 0.    , 0.    ],
        [0.    , 0.    , 0.375 , 0.8125, 0.625 , 0.    , 0.    , 0.    ]],

       [[0.    , 0.    , 0.    , 0.75  , 0.8125, 0.3125, 0.    , 0.    ],
        [0.    , 0.    , 0.    , 0.6875, 1.    , 0.5625, 0.    , 0.    ],
        [0.    , 0.    , 0.1875, 0.9375, 1.    , 0.375 , 0.    , 0.    ],
        [0.    , 0.4375, 0.9375, 1.    , 1.    , 0.125 , 0.    , 0.    ],
        [0.    , 0.    , 0.0625, 1.    , 1.    , 0.1875, 0

In [119]:
inp = tf.keras.layers.Input(shape = img_shape)
flat = tf.keras.layers.Flatten()(inp)
fc1 = tf.keras.layers.Dense(10, activation='relu')(flat)
out = tf.keras.layers.Softmax()(fc1)
model = tf.keras.models.Model(inputs = inp, outputs = out)
print(model.summary())

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 8, 8)]            0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                650       
_________________________________________________________________
softmax_15 (Softmax)         (None, 10)                0         
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________
None


In [120]:
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
optimizer = tf.keras.optimizers.Adam(lr = 0.1, clipnorm = 5)
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  factor=0.1, 
                                                  patience=5, 
                                                  min_lr=0.0001, 
                                                  min_delta=0.01, 
                                                  cooldown=5)]
model.compile(loss = loss, metrics = metrics)

In [121]:
model.fit(dataset,  epochs = 100, validation_data = dataset_val, callbacks= callbacks)

Epoch 1/100
337/337 [==============================] - 1s 2ms/step - loss: 2.2280 - accuracy: 0.2301 - val_loss: 2.1180 - val_accuracy: 0.3000
Epoch 2/100
337/337 [==============================] - 1s 2ms/step - loss: 2.0423 - accuracy: 0.2888 - val_loss: 1.9792 - val_accuracy: 0.3000
Epoch 3/100
337/337 [==============================] - 1s 2ms/step - loss: 1.9398 - accuracy: 0.2918 - val_loss: 1.8967 - val_accuracy: 0.3022
Epoch 4/100
337/337 [==============================] - 1s 2ms/step - loss: 1.8771 - accuracy: 0.2932 - val_loss: 1.8445 - val_accuracy: 0.3022
Epoch 5/100
337/337 [==============================] - 1s 2ms/step - loss: 1.8358 - accuracy: 0.2932 - val_loss: 1.8090 - val_accuracy: 0.3022
Epoch 6/100
337/337 [==============================] - 1s 2ms/step - loss: 1.8066 - accuracy: 0.2947 - val_loss: 1.7837 - val_accuracy: 0.3022
Epoch 7/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7852 - accuracy: 0.2947 - val_loss: 1.7649 - val_accuracy: 0.3044

Epoch 58/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6828 - accuracy: 0.2947 - val_loss: 1.6704 - val_accuracy: 0.3022
Epoch 59/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6827 - accuracy: 0.2947 - val_loss: 1.6703 - val_accuracy: 0.3022
Epoch 60/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6826 - accuracy: 0.2947 - val_loss: 1.6701 - val_accuracy: 0.3022
Epoch 61/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6824 - accuracy: 0.2947 - val_loss: 1.6700 - val_accuracy: 0.3022
Epoch 62/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6823 - accuracy: 0.2947 - val_loss: 1.6699 - val_accuracy: 0.3022
Epoch 63/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6822 - accuracy: 0.2947 - val_loss: 1.6697 - val_accuracy: 0.3022
Epoch 64/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6821 - accuracy: 0.2947 - val_loss: 1.6696 - val_accuracy:

In [112]:
val_loss, val_acc = model.evaluate(dataset_val)

113/113 [==============================] - 0s 1ms/step - loss: 0.6516 - accuracy: 0.7578
[0.6515841484069824, 0.757777750492096]


In [127]:
results = pd.DataFrame(columns = ['fc1_n', 'fc2_n', 'acc', 'loss', 'val_loss', 'val_acc'])
for i in range(10, 50, 10):
    for j in range(10, 40, 10):
        inp = tf.keras.layers.Input(shape = img_shape)
        flat = tf.keras.layers.Flatten()(inp)
        fc1 = tf.keras.layers.Dense(i, activation='relu')(flat)
        fc2 = tf.keras.layers.Dense(j, activation='relu')(fc1)
        fc3 = tf.keras.layers.Dense(n_labels, activation='relu')(fc2)
        out = tf.keras.layers.Softmax()(fc3)
        model = tf.keras.models.Model(inputs = inp, outputs = out)
        model.compile(loss = loss, metrics = metrics)
        model.fit(dataset,  epochs = 100, validation_data = dataset_val, callbacks= callbacks)
        loss_train, acc_train = model.evaluate(dataset)
        val_loss, val_acc = model.evaluate(dataset_val)
        results = results.append({'fc1_n': i,
                                'fc2_n' : j,
                                'acc' : val_acc,
                                'loss': loss_train,
                                'val_loss': val_loss,
                                'val_acc': val_acc}, ignore_index=True)

Epoch 1/100
337/337 [==============================] - 1s 2ms/step - loss: 2.2565 - accuracy: 0.2034 - val_loss: 2.1739 - val_accuracy: 0.2644
Epoch 2/100
337/337 [==============================] - 1s 2ms/step - loss: 2.0361 - accuracy: 0.3177 - val_loss: 1.9192 - val_accuracy: 0.3200
Epoch 3/100
337/337 [==============================] - 1s 2ms/step - loss: 1.8295 - accuracy: 0.3519 - val_loss: 1.7455 - val_accuracy: 0.3578
Epoch 4/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6905 - accuracy: 0.3771 - val_loss: 1.6148 - val_accuracy: 0.3956
Epoch 5/100
337/337 [==============================] - 1s 2ms/step - loss: 1.5921 - accuracy: 0.3853 - val_loss: 1.5380 - val_accuracy: 0.3956
Epoch 6/100
337/337 [==============================] - 1s 2ms/step - loss: 1.5371 - accuracy: 0.3868 - val_loss: 1.4971 - val_accuracy: 0.3956
Epoch 7/100
337/337 [==============================] - 1s 2ms/step - loss: 1.5070 - accuracy: 0.3868 - val_loss: 1.4729 - val_accuracy: 0.3978

337/337 [==============================] - 1s 2ms/step - loss: 1.1968 - accuracy: 0.4900 - val_loss: 1.1972 - val_accuracy: 0.4956
Epoch 58/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1967 - accuracy: 0.4900 - val_loss: 1.1972 - val_accuracy: 0.4956
Epoch 59/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1965 - accuracy: 0.4900 - val_loss: 1.1971 - val_accuracy: 0.4956
Epoch 60/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1964 - accuracy: 0.4900 - val_loss: 1.1970 - val_accuracy: 0.4956
Epoch 61/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1963 - accuracy: 0.4900 - val_loss: 1.1970 - val_accuracy: 0.4956
Epoch 62/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1962 - accuracy: 0.4900 - val_loss: 1.1969 - val_accuracy: 0.4956
Epoch 63/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1960 - accuracy: 0.4900 - val_loss: 1.1968 - val_accuracy: 0.4956
Epoch

Epoch 13/100
337/337 [==============================] - 1s 2ms/step - loss: 0.6050 - accuracy: 0.7780 - val_loss: 0.6416 - val_accuracy: 0.7556
Epoch 14/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5960 - accuracy: 0.7788 - val_loss: 0.6355 - val_accuracy: 0.7556
Epoch 15/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5881 - accuracy: 0.7795 - val_loss: 0.6306 - val_accuracy: 0.7556
Epoch 16/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5815 - accuracy: 0.7817 - val_loss: 0.6269 - val_accuracy: 0.7556
Epoch 17/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5757 - accuracy: 0.7817 - val_loss: 0.6234 - val_accuracy: 0.7556
Epoch 18/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5706 - accuracy: 0.7832 - val_loss: 0.6211 - val_accuracy: 0.7556
Epoch 19/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5656 - accuracy: 0.7832 - val_loss: 0.6186 - val_accuracy:

Epoch 70/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5199 - accuracy: 0.7877 - val_loss: 0.5998 - val_accuracy: 0.7667
Epoch 71/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5198 - accuracy: 0.7877 - val_loss: 0.6002 - val_accuracy: 0.7667
Epoch 72/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5197 - accuracy: 0.7877 - val_loss: 0.6001 - val_accuracy: 0.7667
Epoch 73/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5196 - accuracy: 0.7877 - val_loss: 0.6002 - val_accuracy: 0.7667
Epoch 74/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5195 - accuracy: 0.7884 - val_loss: 0.5996 - val_accuracy: 0.7667
Epoch 75/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5194 - accuracy: 0.7877 - val_loss: 0.6000 - val_accuracy: 0.7667
Epoch 76/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5192 - accuracy: 0.7884 - val_loss: 0.5999 - val_accuracy:

Epoch 26/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7601 - accuracy: 0.7869 - val_loss: 0.8285 - val_accuracy: 0.7533
Epoch 27/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7599 - accuracy: 0.7869 - val_loss: 0.8283 - val_accuracy: 0.7533
Epoch 28/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7596 - accuracy: 0.7869 - val_loss: 0.8282 - val_accuracy: 0.7556
Epoch 29/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7594 - accuracy: 0.7869 - val_loss: 0.8281 - val_accuracy: 0.7556
Epoch 30/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7592 - accuracy: 0.7869 - val_loss: 0.8278 - val_accuracy: 0.7556
Epoch 31/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7589 - accuracy: 0.7869 - val_loss: 0.8278 - val_accuracy: 0.7556
Epoch 32/100
337/337 [==============================] - 1s 2ms/step - loss: 0.7587 - accuracy: 0.7877 - val_loss: 0.8277 - val_accuracy:

...

337/337 [==============================] - 1s 2ms/step - loss: 0.0641 - accuracy: 0.9822 - val_loss: 0.1492 - val_accuracy: 0.9556
Epoch 58/100
337/337 [==============================] - 1s 2ms/step - loss: 0.0639 - accuracy: 0.9822 - val_loss: 0.1494 - val_accuracy: 0.9556
Epoch 59/100
337/337 [==============================] - 1s 2ms/step - loss: 0.0637 - accuracy: 0.9822 - val_loss: 0.1497 - val_accuracy: 0.9556
Epoch 60/100
337/337 [==============================] - 1s 2ms/step - loss: 0.0635 - accuracy: 0.9822 - val_loss: 0.1499 - val_accuracy: 0.9556
Epoch 61/100
337/337 [==============================] - 1s 2ms/step - loss: 0.0633 - accuracy: 0.9822 - val_loss: 0.1501 - val_accuracy: 0.9556
Epoch 62/100
337/337 [==============================] - 1s 2ms/step - loss: 0.0632 - accuracy: 0.9822 - val_loss: 0.1501 - val_accuracy: 0.9556
Epoch 63/100
337/337 [==============================] - 1s 2ms/step - loss: 0.0630 - accuracy: 0.9822 - val_loss: 0.1504 - val_accuracy: 0.9556
Epoch

Epoch 13/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2303 - accuracy: 0.5791 - val_loss: 1.1970 - val_accuracy: 0.5933
Epoch 14/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2264 - accuracy: 0.5791 - val_loss: 1.1977 - val_accuracy: 0.5867
Epoch 15/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2156 - accuracy: 0.5783 - val_loss: 1.1911 - val_accuracy: 0.5889
Epoch 16/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2143 - accuracy: 0.5776 - val_loss: 1.1904 - val_accuracy: 0.5889
Epoch 17/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2139 - accuracy: 0.5776 - val_loss: 1.1899 - val_accuracy: 0.5889
Epoch 18/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2134 - accuracy: 0.5776 - val_loss: 1.1897 - val_accuracy: 0.5889
Epoch 19/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2129 - accuracy: 0.5776 - val_loss: 1.1892 - val_accuracy:

Epoch 70/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2004 - accuracy: 0.5820 - val_loss: 1.1830 - val_accuracy: 0.5933
Epoch 71/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2003 - accuracy: 0.5828 - val_loss: 1.1823 - val_accuracy: 0.5933
Epoch 72/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2001 - accuracy: 0.5828 - val_loss: 1.1824 - val_accuracy: 0.5933
Epoch 73/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1999 - accuracy: 0.5820 - val_loss: 1.1821 - val_accuracy: 0.5933
Epoch 74/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1997 - accuracy: 0.5828 - val_loss: 1.1823 - val_accuracy: 0.5933
Epoch 75/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1997 - accuracy: 0.5820 - val_loss: 1.1816 - val_accuracy: 0.5933
Epoch 76/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1994 - accuracy: 0.5828 - val_loss: 1.1820 - val_accuracy:

337/337 [==============================] - 1s 2ms/step - loss: 1.4048 - accuracy: 0.3935 - val_loss: 1.4250 - val_accuracy: 0.3933
Epoch 26/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4047 - accuracy: 0.3935 - val_loss: 1.4242 - val_accuracy: 0.3933
Epoch 27/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4046 - accuracy: 0.3935 - val_loss: 1.4251 - val_accuracy: 0.3933
Epoch 28/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4045 - accuracy: 0.3935 - val_loss: 1.4248 - val_accuracy: 0.3933
Epoch 29/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4044 - accuracy: 0.3935 - val_loss: 1.4246 - val_accuracy: 0.3911
Epoch 30/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4045 - accuracy: 0.3935 - val_loss: 1.4257 - val_accuracy: 0.3911
Epoch 31/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4043 - accuracy: 0.3935 - val_loss: 1.4250 - val_accuracy: 0.3933
Epoch

Epoch 82/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4029 - accuracy: 0.3942 - val_loss: 1.4320 - val_accuracy: 0.3911
Epoch 83/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4027 - accuracy: 0.3942 - val_loss: 1.4325 - val_accuracy: 0.3911
Epoch 84/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4029 - accuracy: 0.3942 - val_loss: 1.4326 - val_accuracy: 0.3911
Epoch 85/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4027 - accuracy: 0.3942 - val_loss: 1.4332 - val_accuracy: 0.3911
Epoch 86/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4027 - accuracy: 0.3942 - val_loss: 1.4319 - val_accuracy: 0.3911
Epoch 87/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4026 - accuracy: 0.3942 - val_loss: 1.4329 - val_accuracy: 0.3911
Epoch 88/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4027 - accuracy: 0.3942 - val_loss: 1.4326 - val_accuracy:

In [126]:
results

,fc1_n,fc2_n,acc,loss,val_loss,val_acc
0,10.0,10.0,0.733333,0.742614,0.888836,0.733333
1,10.0,20.0,0.668889,0.983362,1.042482,0.668889
2,20.0,10.0,0.495556,1.196243,1.208126,0.495556
3,20.0,20.0,0.593333,1.181800,1.201827,0.593333
4,30.0,10.0,0.677778,0.726991,0.800615,0.677778
5,30.0,20.0,0.562222,1.177453,1.262470,0.562222
6,40.0,10.0,0.597778,0.944834,0.968744,0.597778
7,40.0,20.0,0.493333,1.395849,1.438141,0.493333
